In [1]:
import numpy as np
import pandas as pd
import random
import os

C:\Users\kanch\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


**Step 1: - Helpers**

In [2]:
def check_winner(b):
    for i in range(3):
        if abs(b[i].sum()) == 3: return int(np.sign(b[i].sum()))   # row
        if abs(b[:,i].sum()) == 3: return int(np.sign(b[:,i].sum())) # col
    if abs(np.trace(b)) == 3: return int(np.sign(np.trace(b)))
    if abs(np.trace(np.fliplr(b))) == 3: return int(np.sign(np.trace(np.fliplr(b))))
    if not (b==0).any(): return 0  # draw
    return None

def moves(b): return [(i,j) for i in range(3) for j in range(3) if b[i,j]==0]
def b2s(b): return ''.join(map(str,b.flatten()))

**Step 2 : - Simulate random games  save**

In [3]:
def build_probability_table(filename="tic_tac_toe_prob.csv", games=20000):
    prob = {}
    Xw = Ow = D = 0

    for _ in range(games):
        b = np.zeros((3,3),int); turn = 1; hist=[]
        while True:
            m=random.choice(moves(b)); b[m]=turn
            hist.append((b2s(b.copy()),turn))
            w=check_winner(b)
            if w is not None:
                if w==1: Xw+=1
                elif w==-1: Ow+=1
                else: D+=1
                for s,_ in hist:
                    prob.setdefault(s,{"X":0,"O":0,"D":0})
                    if w==1: prob[s]["X"]+=1
                    elif w==-1: prob[s]["O"]+=1
                    else: prob[s]["D"]+=1
                break
            turn*=-1

    print(f"Results ({games} games): X={Xw}, O={Ow}, Draws={D}")
    df = pd.DataFrame.from_dict(prob,orient="index")
    df.to_csv(filename)
    print(f"Probability table saved to {filename}")

**Step 3 : - Load probability table**

In [4]:
def load_probability_table(filename="tic_tac_toe_prob.csv"):
    df = pd.read_csv(filename, index_col=0)
    prob = df.to_dict(orient="index")
    return prob

**Step 4 : - Agent**

In [5]:
def agent(b,player,prob):
    best=None; score=-1e9 if player==1 else 1e9
    for m in moves(b):
        nb=b.copy(); nb[m]=player; k=b2s(nb)
        if k not in prob: continue
        st=prob[k]; tot=st["X"]+st["O"]+st["D"]
        if not tot: continue
        xp,op=st["X"]/tot,st["O"]/tot
        val=(xp-op) if player==1 else (op-xp)
        if (player==1 and val>score) or (player==-1 and val<score):
            best,score=m,val
    return best or random.choice(moves(b))

**Step 5 : - Demo Game**

In [6]:
def demo_game(prob):
    b=np.zeros((3,3),int); turn=1
    print("\nDemo Game:")
    while True:
        m=agent(b,turn,prob); b[m]=turn; print(b,"\n")
        w=check_winner(b)
        if w is not None:
            print("X Wins!" if w==1 else "O Wins!" if w==-1 else "Draw!")
            break
        turn*=-1

**Step 6 : - Run**

In [7]:
if __name__=="__main__":
    filename="tic_tac_toe_prob.csv"

    # If file does not exist, build it
    if not os.path.exists(filename):
        build_probability_table(filename,games=20000)

    # Load prob table and play demo
    prob = load_probability_table(filename)
    demo_game(prob)


Demo Game:
[[0 0 0]
 [0 1 0]
 [0 0 0]] 

[[ 0  0  0]
 [ 0  1  0]
 [ 0 -1  0]] 

[[ 0  0  0]
 [ 0  1  0]
 [ 1 -1  0]] 

[[ 0 -1  0]
 [ 0  1  0]
 [ 1 -1  0]] 

[[ 1 -1  0]
 [ 0  1  0]
 [ 1 -1  0]] 

[[ 1 -1 -1]
 [ 0  1  0]
 [ 1 -1  0]] 

[[ 1 -1 -1]
 [ 1  1  0]
 [ 1 -1  0]] 

X Wins!
